In [12]:
import pandas as pd
import warnings

warnings.filterwarnings("ignore")

asignacion_1 = 'bases/asignacion/09. Septiembre/base_asignacion_septiembre.xlsx'
asignacion_2 = 'bases/asignacion/10. Octubre/base_asignacion_octubre.xlsx'

In [13]:
df_asig_1 = pd.read_excel(asignacion_1, usecols=['CODIGO', 'ID_FLAG', 'AGENCIA_CORRECTA'])
df_asig_1['CODIGO'] = df_asig_1['CODIGO'].astype('Int64').astype(str).str.zfill(8)
df_asig_1.rename(columns={'ID_FLAG': 'FLAG', 'AGENCIA_CORRECTA': 'AGENCIA'}, inplace=True)

df_asig_2 = pd.read_excel(asignacion_2, usecols=['CODIGO', 'FLAG', 'AGENCIA'])
df_asig_2['CODIGO'] = df_asig_2['CODIGO'].astype('Int64').astype(str).str.zfill(8)

df_asig_1.groupby('CODIGO').agg({'FLAG': 'first'}).reset_index().sort_values('CODIGO', ascending=False, inplace=True)
df_asig_2.groupby('CODIGO').agg({'FLAG': 'first'}).reset_index().sort_values('CODIGO', ascending=False, inplace=True)

In [22]:
df_asig = pd.merge(df_asig_1, df_asig_2, on='CODIGO', how='outer', suffixes=('_1', '_2'))
df_asig['FLAG_1'] = df_asig['FLAG_1'].fillna(0)
df_asig['FLAG_2'] = df_asig['FLAG_2'].fillna(0)
df_asig['FLAG_1'] = df_asig['FLAG_1'].astype(int)
df_asig['FLAG_2'] = df_asig['FLAG_2'].astype(int)

def determine_flag(row):
    if (row['FLAG_1'] == row['FLAG_2']) or (row['FLAG_1'] > 1 and row['FLAG_2'] > 1):
        return 'IGUAL'
    elif row['FLAG_1'] == 1 and row['FLAG_2'] > 1:
        return 'MONO A MULTI'
    elif row['FLAG_1'] > 1 and row['FLAG_2'] == 1:
        return 'MULTI A MONO'
    elif row['FLAG_1'] == 0 and row['FLAG_2'] == 1:
        return 'NO A MONO'
    elif row['FLAG_1'] == 0 and row['FLAG_2'] > 1:
        return 'NO A MULTI'
    elif row['FLAG_1'] > 1 and row['FLAG_2'] == 0:
        return 'MULTI A NO'
    elif row['FLAG_1'] == 1 and row['FLAG_2'] == 0:
        return 'MONO A NO'
    else:
        return 'NO A NO'

def determine_agencia(row):
    if row['AGENCIA_1'] == row['AGENCIA_2']:
        return row['AGENCIA_1']
    else:
        return f"{row['AGENCIA_1']} - {row['AGENCIA_2']}"

df_asig['FLAG'] = df_asig.apply(determine_flag, axis=1)
df_asig['AGENCIA_1'] = df_asig['AGENCIA_1'].fillna('SIN AGENCIA')
df_asig['AGENCIA_2'] = df_asig['AGENCIA_2'].fillna('SIN AGENCIA')
df_asig['AGENCIA'] = df_asig.apply(determine_agencia, axis=1)
df_asig = df_asig[['CODIGO', 'FLAG_1', 'FLAG_2', 'FLAG', 'AGENCIA_1', 'AGENCIA_2', 'AGENCIA']]

In [23]:
df_asig.to_excel('bases/asignacion/base_asignacion_cruce.xlsx', index=False)